In [1]:
import os

In [2]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\LLM-Text-Detection\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\LLM-Text-Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path




@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    target_column: str

In [6]:
from Toxic_comment_detection.utils.common import *
from Toxic_comment_detection.constants import *

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path = config.model_path,
            target_column = schema.Label
        )

        return model_evaluation_config

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    def eval(self,x_test,y_test):
        # Load the model from the specified path
        model = load_model(self.config.model_path)
        
        # Evaluate the model on the test set
        loss, accuracy = model.evaluate(x_test,y_test)
        logger.info(f'Test Accuracy: {accuracy}')
        
        # Generate predictions on the test set
        y_pred_proba = model.predict(self.config.x_test)
        y_pred = (y_pred_proba > 0.5).astype(int)
        
        # Generate classification report
        report = classification_report(y_test,y_pred)
        print(report)

        # Generate classification report for visualization
        report_dict = classification_report(self.config.y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report_dict).transpose()
        
        # Extract relevant metrics for visualization
        class_metrics = df_report.drop(['accuracy', 'macro avg', 'weighted avg'])
        
        # Plotting the classification report metrics using a heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(class_metrics[['precision', 'recall', 'f1-score']], annot=True, cmap='coolwarm', fmt=".2f")
        plt.title('Classification Report Metrics')
        plt.xlabel('Metrics')
        plt.ylabel('Class')
        plt.yticks(rotation=0)
        plt.show()